# Import and preprocess rideshare data. Save to csv for future use.

In [1]:
import pandas as pd
import numpy as np
import math
import os
import glob
import random
import datetime
import shapefile
import geopandas as gpd
import matplotlib.pyplot as plt
import mapclassify

In [2]:
os.chdir(r'C:\Users\stevedc\Documents\DataMining\High Volume FHV Trip Records')
extension = 'csv'
hvall_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#define datatypes, saves memory and hassle later on.
hvdtypes = {'hvfhs_license_num':object, 'dispatching_base_num':object,'pickup_datetime':object,'dropoff_datetime':object,'PULocationID':int,'DOLocationID':int,'SR_Flag':float}

#combine all files in the list
hvdataResolution = 0.01 # defines the fraction of rows to keep
hvcombined_csv = pd.concat([pd.read_csv(f, dtype=hvdtypes, na_values=None, skiprows=lambda i: i>0 and random.random() > hvdataResolution, low_memory=False) for f in hvall_filenames]) # append files, sampling each file before appending according to data resolution

#replace values and convert type
hvcombined_csv.replace('NA', None)

#apply date formatting and add month, day, year columns
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

hvcombined_csv['pickup_datetime'] = pd.to_datetime(hvcombined_csv['pickup_datetime'])
hvcombined_csv['dropoff_datetime'] = pd.to_datetime(hvcombined_csv['dropoff_datetime'])
hvcombined_csv['year'] = hvcombined_csv['pickup_datetime'].dt.year
hvcombined_csv['month'] = hvcombined_csv['pickup_datetime'].dt.month
hvcombined_csv['day'] = hvcombined_csv['pickup_datetime'].dt.day
hvcombined_csv['date'] = hvcombined_csv['pickup_datetime'].dt.date
hvcombined_csv.round(3)

hvzones = r'C:\Users\stevedc\Documents\DataMining\taxi+_zone_lookup.csv'

# Read taxi zones to merge with ty taxi data
hv_zonedtypes = {'Zone':object, 'service_zone':object,'Borough':object}
df_hvzones = pd.read_csv(hvzones, na_values=None, dtype=hv_zonedtypes, delimiter=',')

# merge pickup and dropoff zones with taxi data using LocationID in zones file to match wth PU/DO LocationID in nytaxi file
df_hvaddPUZones = pd.merge(hvcombined_csv, df_hvzones, left_on=['PULocationID'], right_on=['LocationID']).drop(['LocationID'], axis=1)
df_hvaddPUZones.rename(columns={'Borough':'PUBorough', 'Zone':'PUZone', 'service_zone':'PUservice_zone'}, inplace=True) # rename can be combined with merge
df_hvaddDOZones = pd.merge(df_hvaddPUZones, df_hvzones, left_on=['DOLocationID'], right_on=['LocationID']).drop(['LocationID'], axis=1)
df_hvaddDOZones.rename(columns={'Borough':'DOBorough', 'Zone':'DOZone', 'service_zone':'DOservice_zone'}, inplace=True)
#df_hvaddDOZones[['PUBorough', 'PUZone', 'PUservice_zone', 'DOBorough', 'DOZone', 'DOservice_zone']] = df_hvaddDOZones[['PUBorough', 'PUZone', 'PUservice_zone', 'DOBorough', 'DOZone', 'DOservice_zone']].astype(object)

#add covid boolean column
covidDate = '2020-03-13'
df_hvaddDOZones['covid'] = np.where(df_hvaddDOZones['date']<pd.to_datetime(covidDate),False,True)

# clean junk dates, not much data here to clean, can at least extract trip counts
df_hvaddDOZones = df_hvaddDOZones.loc[df_hvaddDOZones['date'] >= pd.to_datetime('2019-01-01')]
df_hvaddDOZones = df_hvaddDOZones.loc[df_hvaddDOZones['date'] <= pd.to_datetime('2020-07-01')]


In [3]:
#expor to csv
hvOutputFile = r'C:\Users\stevedc\Documents\DataMining\Output Files\hv_combined_sampled_1pcnt.csv'
df_hvaddDOZones.to_csv(hvOutputFile, index=False)

In [ ]:
df_hvaddDOZones.dtypes

hvfhs_license_num               object
dispatching_base_num            object
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int32
DOLocationID                     int32
SR_Flag                        float64
year                             int64
month                            int64
day                              int64
date                            object
PUBorough                       object
PUZone                          object
PUservice_zone                  object
DOBorough                       object
DOZone                          object
DOservice_zone                  object
covid                             bool
dtype: object

In [ ]:
hvfile1 = r'C:\Users\stevedc\Documents\DataMining\Output Files\hv_combined_sampled_1pcnt.csv'
#hv_dtypes = {'hvfhs_license_num':object, 'dispatching_base_num':object,'PULocationID':int32,'DOLocationID':int32,'SR_Flag':float64,'year':int64,'month':int64,'day':int64,'date':object,'PUBorough':object,'PUZone':object,'PUservice_zone':object,'DOBorough':object,'DOZone':object,'DOservice_zone':object,'covid':bool}
#combined_csv['tpep_pickup_datetime'] = pd.to_datetime(combined_csv['tpep_pickup_datetime'])
#combined_csv['tpep_dropoff_datetime'] = pd.to_datetime(combined_csv['tpep_dropoff_datetime'])

df_hvsample = pd.read_csv(hvfile1, delimiter=',', dtype=hv_dtypes)
df_hvsample.head()
